In [1]:
import os
import numpy as np
import pandas as pd
from typing import Any

from exp_spec_info import *
from plot_info import *
from select_data import *

In [2]:
# Data pickle path
extracted_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\extracted_data.pkl"

# Processed data paths
extended_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\extended_data.pkl"
median_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\median_data.pkl"

##### Extend Data with Processed Elements

In [3]:
data = pd.read_pickle(extracted_data_path)

# Extend with final relative residual
rel_res_ext = []
for setup in SETUPS:
    for matrix in SETUP_MATRIX_MAPPING[setup]:
        for restart_param in RESTART_PARAMS:
            for exp_iter in range(N_EXPERIMENT_ITERATIONS):

                sub_data = df_sel_setup_matrix_restart(
                    data, setup, matrix, restart_param
                )
                sub_data = sub_data[sub_data["experiment_iter"] == exp_iter]

                rel_res = sub_data["inner_relres"].apply(
                    lambda relres_arr: relres_arr[-1]
                )
                if len(rel_res) > 0:
                    rel_res_ext.append(rel_res)

rel_res_ext = pd.concat(rel_res_ext)
rel_res_ext.name = "final_relres"

extended_data = data.join((rel_res_ext))


def rel_res_frac_err(target_relres: Any, baseline_relres: Any) -> Any:
    return (target_relres-baseline_relres)/baseline_relres


def mean_rel_res_frac_err(
    target_relres: np.array, baseline_relres: np.array
) -> float:
    low_n = min(len(target_relres), len(baseline_relres))
    return np.mean(
        rel_res_frac_err(target_relres[:low_n], baseline_relres[:low_n])
    )


# Extend with values based on FP32 and FP64
rel_time_fp64_ext = []
rel_res_frac_err_fp64_ext = []
outer_mean_rel_res_frac_err_fp64_ext = []
inner_mean_rel_res_frac_err_fp64_ext = []

rel_time_fp32_ext = []
rel_res_frac_err_fp32_ext = []
outer_mean_rel_res_frac_err_fp32_ext = []
inner_mean_rel_res_frac_err_fp32_ext = []

for setup in SETUPS:
    for matrix in SETUP_MATRIX_MAPPING[setup]:
        for restart_param in RESTART_PARAMS:
            for exp_iter in range(N_EXPERIMENT_ITERATIONS):

                sub_data = df_sel_setup_matrix_restart(
                    extended_data, setup, matrix, restart_param
                )
                sub_data = sub_data[sub_data["experiment_iter"] == exp_iter]

                if sub_data[sub_data["solver"] == "FP FP64"].size != 0:

                    fp64_row = sub_data[sub_data["solver"] == "FP FP64"].iloc[0]

                    rel_time_fp64_ext.append(
                        sub_data["elapsed_time_ms"]/fp64_row["elapsed_time_ms"]
                    )

                    rel_res_frac_err_fp64_ext.append(
                        rel_res_frac_err(
                            sub_data["final_relres"], fp64_row["final_relres"]
                        )
                    )

                    series_mean_relres_frac_err = sub_data["outer_relres"].transform(
                            mean_rel_res_frac_err,
                            0,
                            fp64_row["outer_relres"]
                        )
                    assert series_mean_relres_frac_err[
                        sub_data["solver"] == "FP FP64"
                    ].values[0] == 0.
                    outer_mean_rel_res_frac_err_fp64_ext.append(
                        series_mean_relres_frac_err
                    )

                    series_mean_relres_frac_err = sub_data["inner_relres"].transform(
                            mean_rel_res_frac_err,
                            0,
                            fp64_row["inner_relres"]
                        )
                    assert series_mean_relres_frac_err[
                        sub_data["solver"] == "FP FP64"
                    ].values[0] == 0.
                    inner_mean_rel_res_frac_err_fp64_ext.append(
                        series_mean_relres_frac_err
                    )
                

                if sub_data[sub_data["solver"] == "FP FP32"].size != 0:

                    fp32_row = sub_data[sub_data["solver"] == "FP FP32"].iloc[0]

                    rel_time_fp32_ext.append(
                        sub_data["elapsed_time_ms"]/fp32_row["elapsed_time_ms"]
                    )

                    rel_res_frac_err_fp32_ext.append(
                        rel_res_frac_err(
                            sub_data["final_relres"], fp32_row["final_relres"]
                        )
                    )

                    series_mean_relres_frac_err = sub_data["outer_relres"].transform(
                            mean_rel_res_frac_err,
                            0,
                            fp32_row["outer_relres"]
                        )
                    assert series_mean_relres_frac_err[
                        sub_data["solver"] == "FP FP32"
                    ].values[0] == 0.
                    outer_mean_rel_res_frac_err_fp32_ext.append(
                        series_mean_relres_frac_err
                    )

                    series_mean_relres_frac_err = sub_data["inner_relres"].transform(
                            mean_rel_res_frac_err,
                            0,
                            fp32_row["inner_relres"]
                        )
                    assert series_mean_relres_frac_err[
                        sub_data["solver"] == "FP FP32"
                    ].values[0] == 0.
                    inner_mean_rel_res_frac_err_fp32_ext.append(
                        series_mean_relres_frac_err
                    )

rel_time_fp64_ext = pd.concat(rel_time_fp64_ext)
rel_time_fp64_ext.name = "rel_time_fp64"

rel_res_frac_err_fp64_ext = pd.concat(rel_res_frac_err_fp64_ext)
rel_res_frac_err_fp64_ext.name = "rel_res_frac_err_fp64"

outer_mean_rel_res_frac_err_fp64_ext = pd.concat(outer_mean_rel_res_frac_err_fp64_ext)
outer_mean_rel_res_frac_err_fp64_ext.name = "outer_mean_rel_res_frac_err_fp64"

inner_mean_rel_res_frac_err_fp64_ext = pd.concat(inner_mean_rel_res_frac_err_fp64_ext)
inner_mean_rel_res_frac_err_fp64_ext.name = "inner_mean_rel_res_frac_err_fp64"

rel_time_fp32_ext = pd.concat(rel_time_fp32_ext)
rel_time_fp32_ext.name = "rel_time_fp32"

rel_res_frac_err_fp32_ext = pd.concat(rel_res_frac_err_fp32_ext)
rel_res_frac_err_fp32_ext.name = "rel_res_frac_err_fp32"

outer_mean_rel_res_frac_err_fp32_ext = pd.concat(outer_mean_rel_res_frac_err_fp32_ext)
outer_mean_rel_res_frac_err_fp32_ext.name = "outer_mean_rel_res_frac_err_fp32"

inner_mean_rel_res_frac_err_fp32_ext = pd.concat(inner_mean_rel_res_frac_err_fp32_ext)
inner_mean_rel_res_frac_err_fp32_ext.name = "inner_mean_rel_res_frac_err_fp32"

extended_data = extended_data.join(
    (
        rel_time_fp64_ext,
        rel_res_frac_err_fp64_ext,
        outer_mean_rel_res_frac_err_fp64_ext,
        inner_mean_rel_res_frac_err_fp64_ext,
        rel_time_fp32_ext,
        rel_res_frac_err_fp32_ext,
        outer_mean_rel_res_frac_err_fp32_ext,
        inner_mean_rel_res_frac_err_fp32_ext
    )
)
display(extended_data)

extended_data.to_pickle(extended_data_path)

,setup,matrix,solver,restart_param,experiment_iter,populated,initiated,converged,terminated,outer_iters,...,inner_relres,final_relres,rel_time_fp64,rel_res_frac_err_fp64,outer_mean_rel_res_frac_err_fp64,inner_mean_rel_res_frac_err_fp64,rel_time_fp32,rel_res_frac_err_fp32,outer_mean_rel_res_frac_err_fp32,inner_mean_rel_res_frac_err_fp32
0,ilu0,af23560,FP FP16,10,0,True,True,False,True,1500,...,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000e+00,0.274509,-0.965365,-0.964723,-0.963196,0.296736,-0.573871,-0.585890,-0.630190
1,ilu0,af23560,FP FP16,10,1,True,True,False,True,1500,...,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000e+00,0.273178,-0.972670,-0.972022,-0.971219,0.294984,-0.648679,-0.656811,-0.658723
2,ilu0,af23560,FP FP16,10,2,True,True,False,True,1500,...,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000e+00,0.272790,-0.976667,-0.976017,-0.974570,0.294943,-0.857800,-0.855115,-0.839547
3,ilu0,af23560,FP FP16,20,0,True,True,False,True,750,...,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000e+00,0.173880,-0.994418,-0.993098,-0.990586,0.185687,-0.599894,-0.599795,-0.590696
4,ilu0,af23560,FP FP16,20,1,True,True,False,True,750,...,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000e+00,0.173014,-0.993818,-0.992539,-0.990827,0.185035,-0.874479,-0.853338,-0.824724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36475,unprecond,Zhao2,PC HSD S2T,150,1,True,True,True,True,27,...,"[1.0, 0.9798814359645774, 0.4454125543127461, ...",1.000000e-12,1.031818,0.000000,2.910146,0.787360,1.195662,0.000000,2.964854,0.807149
36476,unprecond,Zhao2,PC HSD S2T,150,2,True,True,True,True,27,...,"[1.0, 0.9808629620107184, 0.4513029714477763, ...",1.000000e-12,1.033342,0.000000,2.000821,0.398173,1.192739,0.000000,1.999751,0.397897
36477,unprecond,Zhao2,PC HSD S2T,200,0,True,True,True,True,20,...,"[1.0, 0.9766142994493332, 0.44873103183878926,...",1.000000e-12,1.191292,0.000000,56.320553,40.998211,1.384143,0.000000,56.418149,41.025403
36478,unprecond,Zhao2,PC HSD S2T,200,1,True,True,True,True,19,...,"[1.0, 0.9779367541164282, 0.4471612329439548, ...",1.000000e-12,1.137896,0.000000,12.832702,9.363653,1.314655,0.000000,12.824701,9.349476


##### Compress into Experiment Iteration Median Values

In [4]:
median_data = []

for setup in SETUPS:
    for matrix in SETUP_MATRIX_MAPPING[setup]:
        for restart_param in RESTART_PARAMS:

            sub_data = df_sel_setup_matrix_restart(
                extended_data, setup, matrix, restart_param
            )

            for solver in SOLVERS:

                solve_experiments = sub_data[sub_data["solver"] == solver]

                if len(solve_experiments) > 0:

                    median_experiment = {
                        "setup": setup,
                        "matrix": matrix,
                        "solver": solver,
                        "restart_param": restart_param,
                        "all_populated": bool(solve_experiments["populated"].all()),
                        "all_initiated": bool(solve_experiments["initiated"].all()),
                        "all_converged": bool(solve_experiments["converged"].all()),
                        "all_terminated": bool(solve_experiments["terminated"].all()),
                        "med_outer_iter": int(solve_experiments["outer_iters"].median()),
                        "med_inner_iter": int(solve_experiments["inner_iters"].median()),
                        "med_time_ms": int(solve_experiments["elapsed_time_ms"].median()),
                        "med_rel_res": solve_experiments["final_relres"].median(),
                        "med_rel_time_fp64": solve_experiments["rel_time_fp64"].median(),
                        "med_rel_res_frac_err_fp64": solve_experiments["rel_res_frac_err_fp64"].median(),
                        "med_outer_mean_rel_res_frac_err_fp64": (
                            solve_experiments["outer_mean_rel_res_frac_err_fp64"].median()
                        ),
                        "med_inner_mean_rel_res_frac_err_fp64": (
                            solve_experiments["inner_mean_rel_res_frac_err_fp64"].median()
                        ),
                        "med_rel_time_fp32": solve_experiments["rel_time_fp32"].median(),
                        "med_rel_res_frac_err_fp32": solve_experiments["rel_res_frac_err_fp32"].median(),
                        "med_outer_mean_rel_res_frac_err_fp32": (
                            solve_experiments["outer_mean_rel_res_frac_err_fp32"].median()
                        ),
                        "med_inner_mean_rel_res_frac_err_fp32": (
                            solve_experiments["inner_mean_rel_res_frac_err_fp32"].median()
                        )
                    }

                    median_data.append(median_experiment)

median_data = pd.DataFrame(median_data)
display(median_data)

median_data.to_pickle(median_data_path)

,setup,matrix,solver,restart_param,all_populated,all_initiated,all_converged,all_terminated,med_outer_iter,med_inner_iter,med_time_ms,med_rel_res,med_rel_time_fp64,med_rel_res_frac_err_fp64,med_outer_mean_rel_res_frac_err_fp64,med_inner_mean_rel_res_frac_err_fp64,med_rel_time_fp32,med_rel_res_frac_err_fp32,med_outer_mean_rel_res_frac_err_fp32,med_inner_mean_rel_res_frac_err_fp32
0,unprecond,af_0_k101,FP FP16,10,True,True,False,True,1500,15000,159474,0.000013,0.993768,1.344918e+00,0.647142,0.642290,1.304469,1.405960,0.656784,0.651889
1,unprecond,af_0_k101,FP FP32,10,True,True,False,True,1500,15000,122252,0.000005,0.761818,-2.537117e-02,-0.004336,-0.004331,1.000000,0.000000,0.000000,0.000000
2,unprecond,af_0_k101,FP FP64,10,True,True,False,True,1500,15000,160457,0.000005,1.000000,0.000000e+00,0.000000,0.000000,1.312649,0.026032,0.004403,0.004398
3,unprecond,af_0_k101,PC SD ORC,10,True,True,False,True,1500,15000,137222,0.000005,0.855195,-9.946886e-03,-0.002348,-0.002346,1.123272,0.015826,0.002023,0.002019
4,unprecond,af_0_k101,PC SD CS,10,True,True,False,True,1500,15000,121514,0.000005,0.760301,-2.537117e-02,-0.004336,-0.004331,0.999301,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12155,ilutp1em4,Zhao2,PC SD RRT,200,True,True,False,True,75,75,5111,1.000000,0.015808,-2.296659e-01,-0.226644,-0.302835,1.003929,0.000000,0.000000,0.000000
12156,ilutp1em4,Zhao2,PC HSD ORC,200,True,True,False,True,75,6045,133032,1.298138,0.411441,-9.683974e-13,-0.135986,-0.001305,26.099882,0.298138,0.117686,0.201745
12157,ilutp1em4,Zhao2,PC HSD CS,200,True,True,False,True,75,14602,315072,1.298138,0.974522,-5.108851e-13,-0.006044,0.000030,61.827316,0.298138,0.286369,0.444045
12158,ilutp1em4,Zhao2,PC HSD RRT,200,True,True,False,True,75,75,7828,1.000000,0.024213,-2.296659e-01,-0.226644,-0.302835,1.536107,0.000000,0.000000,0.000000
